# Dashboard

# ESMR data from open data
https://data.ca.gov/dataset/water-quality-effluent-electronic-self-monitoring-report-esmr-data

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import panel as pn
import param
import holoviews as hv
from holoviews import opts, dim
import hvplot.pandas
import numpy as np
import pandas as pd
import os
import warnings
warnings.filterwarnings("ignore")
# viz libs
hv.extension('bokeh')
pn.extension('tabulator')

## Read in data

In [3]:
# needs to be done once
#!pip install -e ../

In [4]:
esmr_file = '../tests/data/esmr-analytical-export_years-2006-2024_2024-03-13.csv'
#esmr_file = '../tests/data/esmr-analytical-export_year-2024_2024-12-02.csv'


In [ ]:
from esmr_data import esmr
df = esmr.read_data_csv(esmr_file)
data = esmr.ESMR(df)
print('Number of WWTP facilities : ', len(data.get_facility_names()))

## Plot for facilities that have location information of lat/lon

In [ ]:
df.head()

In [ ]:
facility_location_lat_lon = esmr.build_facility_location_lat_lon(df)
facility_location_lat_lon.head()

## Many facilities don't have lat/lon !

In [ ]:
facility_no_lat_lon = esmr.get_facilities_with_no_latlon_info(
    df, facility_location_lat_lon)
facility_no_lat_lon.head()

# Extract select facilities and parameters (flow, ec, temp)

In [9]:
facility_names = ['EchoWater Resource Recovery Facility', 'Mountain House WWTP', 'Tracy WWTP', 'City of Manteca WW Quality Control Facility', 'Stockton Regional WW Control Facility', 
                  'White Slough Water Pollution Control Facility', 'Ironhouse WWTF', 'Sac City Combined WW Collection/TRT Sys', 'Brentwood WWTP']
parameters = ['Flow', 'Temperature', 'Electrical Conductivity @ 25 Deg. C']
plots = {'Flow': [], 'Temperature': [], 'Electrical_Conductivity_@_25_Deg._C': []}
dfmap = {}
for facility_name in facility_names:
    location_place_type = 'Effluent Monitoring'
    for parameter in parameters:
        dff = df[(df.facility_name == facility_name) & (df.location_place_type == 'Effluent Monitoring') & (df.parameter == parameter)]
        fname = facility_name.replace(' ', '_')
        fname = fname.replace('/', '_')
        pname = parameter.replace(' ', '_')
        plots[pname].append(dff.hvplot(x='analysis_datetime', y='result', title=f'{facility_name} {parameter}', ylabel=parameter, xlabel='Date', legend=False))
        dfmap[f'{fname}_{pname}'] = dff
        dff.to_csv(f'{fname}_{pname}.csv')

In [10]:
def get_columns_unique_vals(df):
    col_vals = {}
    for col in df.columns:
        if col not in ['result','sampling_datetime','analysis_datetime', 'report_name', 'smr_document_id']:
            col_vals[col]=df[col].unique().tolist()
    return col_vals

In [ ]:
{'EchoWater_Resource_Recovery_Facility_Flow': ('analytical_method','Nan'),
 'EchoWater_Resource_Recovery_Facility_Temperature': ('calculated_method', 'Daily Average (Mean)'),
 'EchoWater_Resource_Recovery_Facility_Electrical_Conductivity_@_25_Deg._C': ('analytical_method', 'Nan'),
 'Mountain_House_WWTP_Flow': ('analytical_method', 'Nan'),
 'Mountain_House_WWTP_Temperature': ('calculated_method', 'Daily Average (Mean)'),
 'Mountain_House_WWTP_Electrical_Conductivity_@_25_Deg._C': ('analytical_method', 'Nan'),}

In [23]:
def write_out_data(df,key):
    if key.endswith('Flow'):
        dfout = df[['result']].resample('D').sum()
    else:
        dfout = df[['result']].resample('D').mean()
    with open(f'raw_{key}.csv', 'w', newline='') as f:
        for ckey, cval in get_columns_unique_vals(df).items():
            f.write(f'# {ckey}: {cval}\n')
        dfout.to_csv(f)

In [ ]:
k='EchoWater_Resource_Recovery_Facility_Flow'
dfk = dfmap[k][dfmap[k].analytical_method.notna()]
write_out_data(dfk,k)
dfk['result'].hvplot()


In [ ]:
k='EchoWater_Resource_Recovery_Facility_Temperature'
dfmap[k][dfmap[k].calculated_method=='Daily Average (Mean)']['result'].hvplot()

In [ ]:
k='EchoWater_Resource_Recovery_Facility_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code=='E120.1']['result'].hvplot()

In [ ]:
k='Mountain_House_WWTP_Flow'
dfmap[k][dfmap[k].analytical_method_code=='DU']['result'].hvplot()

In [ ]:
k='Mountain_House_WWTP_Temperature'
dfmap[k][dfmap[k].calculated_method=='Daily Average (Mean)']['result'].hvplot()

In [ ]:
k='Mountain_House_WWTP_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot.step()

In [ ]:
k='Tracy_WWTP_Flow'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='Tracy_WWTP_Temperature'
dfmap[k][dfmap[k].calculated_method=='Daily Average (Mean)']['result'].hvplot()

In [ ]:
k='Tracy_WWTP_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='City_of_Manteca_WW_Quality_Control_Facility_Flow'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='City_of_Manteca_WW_Quality_Control_Facility_Temperature'
dfmap[k][dfmap[k].calculated_method=='Daily Average (Mean)']['result'].hvplot()

In [ ]:
k='City_of_Manteca_WW_Quality_Control_Facility_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot.step()

In [ ]:
k='Stockton_Regional_WW_Control_Facility_Flow'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='Stockton_Regional_WW_Control_Facility_Temperature'
#dfmap[k][dfmap[k].calculated_method=='Daily Average (Mean)'].hvplot.table()
dfmap[k][dfmap[k].calculated_method=='Daily Average (Mean)']['result'].hvplot()

In [ ]:
k='Stockton_Regional_WW_Control_Facility_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='Brentwood_WWTP_Flow'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='Brentwood_WWTP_Temperature'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='Brentwood_WWTP_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='White_Slough_Water_Pollution_Control_Facility_Flow'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

In [ ]:
k='White_Slough_Water_Pollution_Control_Facility_Temperature'
dfmap[k][dfmap[k].calculated_method=='24-hour Average']['result'].hvplot()

In [ ]:
k='White_Slough_Water_Pollution_Control_Facility_Electrical_Conductivity_@_25_Deg._C'
dfmap[k][dfmap[k].analytical_method_code.notna()]['result'].hvplot()

# Show map of facilities

In [ ]:
map_pts = facility_location_lat_lon.hvplot.points(x='longitude', y='latitude',
                                                  geo=True, tiles=True,
                                                  hover_cols=['facility_name', 'location', 'location_desc'], tools=['tap'],
                                                  nonselection_color='gray', nonselection_alpha=0.5, size=40,
                                                  ).opts(
    frame_width=500,
    active_tools=['wheel_zoom', 'pan', 'tap'],
    title='Facilities with Lat/Lon information')


In [66]:
from esmr_data import dash

In [ ]:
dash.ESMRDash(data).show()